<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/AI_Agent_with_Grok_4_Tool_Calling_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install xai-sdk -q

In [13]:
import json
import re
import os # For environment variable check, if not using Colab userdata

# --- XAI SDK Integration ---
try:
    from xai_sdk import Client
    from xai_sdk.chat import user, system # Keep import as per reference
    from google.colab import userdata
    XAI_key = userdata.get('XAI_KEY')
    print("XAI SDK and Colab userdata imported successfully.")
except ImportError:
    print("Could not import xai_sdk or google.colab.userdata. Please ensure you have xai_sdk installed (`pip install xai_sdk`) and are running in a Colab environment with your XAI_KEY set.")
    print("Falling back to environment variable for XAI_KEY if available.")
    XAI_key = os.environ.get('XAI_KEY')
    if not XAI_key:
        print("XAI_KEY not found in Colab userdata or environment variables. Grok 4 integration will not work.")
        XAI_key = None # Explicitly set to None if not found

client = None
chat = None
if XAI_key:
    try:
        # 1. Initialize the client
        client = Client(
            api_host="api.x.ai",
            api_key=XAI_key
        )
        # 2. Create a chat session with the Grok 4 model
        # Using a temperature of 0 for deterministic tool calling
        chat = client.chat.create(model="grok-4-0709", temperature=0)
        print("XAI Client and Grok 4 chat session initialized.")
    except Exception as e:
        print(f"Error initializing XAI Client or chat session: {e}")
        client = None
        chat = None
else:
    print("XAI_KEY is missing. Grok 4 integration will be skipped. Using simulated LLM.")


# --- LLM Reasoning (Now powered by Grok 4 or simulated fallback) ---
def llm_reasoning_with_grok(prompt):
    """
    Uses Grok 4 to determine the user's intent and required tool/parameters.
    If Grok 4 is not available, falls back to a simple simulated logic.
    """
    if chat and client:
        print(f"\nLLM Reasoning: Sending prompt to Grok 4: '{prompt}'")
        try:
            # Construct the prompt for Grok 4 to encourage structured JSON output
            grok_prompt = f"""
            You are an AI assistant that helps determine which tool to use based on a user's query.
            Your available tools are:
            1.  `get_flight_status`: Use this tool when the user asks about flight status, delays, or flight information.
                Parameters:
                - `flight_number`: (string, required) The flight identifier (e.g., "AC123", "UA456").
            2.  `get_weather_forecast`: Use this tool when the user asks about weather, forecast, or conditions for a specific location.
                Parameters:
                - `location`: (string, required) The city name (e.g., "Montreal", "Toronto").

            If a tool is identified, respond ONLY with a JSON object containing "tool" (string) and "parameters" (object).
            If no specific tool is needed, respond ONLY with a JSON object containing "tool": null and a "response" (string) message.

            Example 1:
            User: What is the status of flight AC123?
            Response: {{"tool": "get_flight_status", "parameters": {{"flight_number": "AC123"}}}}

            Example 2:
            User: What's the weather like in Montreal?
            Response: {{"tool": "get_weather_forecast", "parameters": {{"location": "Montreal"}}}}

            Example 3:
            User: Hello, how are you?
            Response: {{"tool": null, "response": "Hello! I'm doing well, thank for asking. How can I assist you today?"}}

            Now, process the following user query:
            User: {prompt}
            Response:
            """
            # Send the prompt to Grok 4 using the correct method as per reference
            chat.append(user(grok_prompt)) # Append user message
            response = chat.sample() # Sample response

            grok_response_text = response.content # Access the content

            print(f"Grok 4 Raw Response: {grok_response_text}")

            # Attempt to parse the JSON response from Grok 4
            try:
                # Clean up potential unwanted characters before parsing
                grok_response_text = grok_response_text.strip()
                parsed_response = json.loads(grok_response_text)
                if "tool" in parsed_response and ("parameters" in parsed_response or "response" in parsed_response):
                    print(f"Grok 4 parsed successfully. Tool: {parsed_response.get('tool')}")
                    return parsed_response
                else:
                    print("Grok 4 response missing expected keys. Falling back to simulated logic.")
                    return _simulated_llm_reasoning_fallback(prompt)
            except json.JSONDecodeError:
                print(f"Grok 4 response was not valid JSON: {grok_response_text}. Falling back to simulated logic.")
                return _simulated_llm_reasoning_fallback(prompt)

        except Exception as e:
            print(f"Error calling Grok 4 API: {e}. Falling back to simulated logic.")
            return _simulated_llm_reasoning_fallback(prompt)
    else:
        print("Grok 4 client not initialized. Using simulated LLM fallback.")
        return _simulated_llm_reasoning_fallback(prompt)

def _simulated_llm_reasoning_fallback(prompt):
    """
    Fallback function if Grok 4 is not available or fails.
    This is the original simulated logic.
    """
    print(f"Simulated LLM Fallback: Analyzing prompt: '{prompt}'")
    prompt_lower = prompt.lower()

    if "flight status" in prompt_lower or "delay" in prompt_lower or "flight" in prompt_lower:
        match = re.search(r'flight (\w+\d+)', prompt_lower)
        flight_number = match.group(1).upper() if match else "UNKNOWN"
        return {
            "tool": "get_flight_status",
            "parameters": {"flight_number": flight_number},
            "reasoning": f"Simulated: User asking about flight status for flight {flight_number}."
        }
    elif "weather" in prompt_lower or "forecast" in prompt_lower or "conditions" in prompt_lower:
        match = re.search(r'weather (?:like|in|for) (\w+)', prompt_lower)
        location = match.group(1).capitalize() if match else "Montreal" # Default to Montreal as per user's location memory
        return {
            "tool": "get_weather_forecast",
            "parameters": {"location": location},
            "reasoning": f"Simulated: User asking for weather forecast in {location}."
        }
    else:
        # A more generic fallback response for general queries, rather than forcing tool use
        return {
            "tool": None,
            "response": "I'm sorry, I can only provide flight status or weather information at the moment. How else can I assist you regarding those topics?"
        }


# --- Simulated Tools ---
def get_flight_status(flight_number):
    """
    Simulates an API call to get flight status.
    In a real scenario, this would hit a flight tracking API.
    """
    print(f"\nTool Call: Calling 'get_flight_status' for flight: {flight_number}")
    # Mock data based on a few flight numbers
    if flight_number.upper() == "AC123":
        return {
            "flight_number": "AC123",
            "status": "On Time",
            "departure": "Montreal (YUL)",
            "arrival": "Toronto (YYZ)",
            "scheduled_departure_time": "10:00 EST",
            "actual_departure_time": "10:00 EST"
        }
    elif flight_number.upper() == "UA456":
        return {
            "flight_number": "UA456",
            "status": "Delayed",
            "departure": "New York (JFK)",
            "arrival": "Chicago (ORD)",
            "scheduled_departure_time": "14:30 EST",
            "estimated_departure_time": "16:00 EST",
            "delay_reason": "Air Traffic Control"
        }
    else:
        return {"error": f"Flight {flight_number} not found or no status available."}

def get_weather_forecast(location):
    """
    Simulates an API call to get weather forecast.
    In a real scenario, this would hit a weather API.
    """
    print(f"\nTool Call: Calling 'get_weather_forecast' for location: {location}")
    # Mock data
    location_lower = location.lower()
    if location_lower == "montreal":
        return {
            "location": "Montreal, QC",
            "temperature": "22°C",
            "conditions": "Partly Cloudy",
            "humidity": "65%",
            "wind": "15 km/h E",
            "forecast": "Clear skies tonight. Chance of showers tomorrow."
        }
    elif location_lower == "toronto":
        return {
            "location": "Toronto, ON",
            "temperature": "25°C",
            "conditions": "Sunny",
            "humidity": "50%",
            "wind": "10 km/h NW",
            "forecast": "Continued sunny and warm."
        }
    else:
        return {"error": f"Weather for {location} not available."}

# --- Agent Orchestration ---
def ai_agent(task):
    """
    The main AI agent function that orchestrates reasoning and tool calling.
    """
    print(f"\nAgent: Received Task: '{task}'")

    # 1. LLM Reasoning (now using Grok 4 or fallback)
    llm_output = llm_reasoning_with_grok(task)

    # 2. Tool Calling based on LLM's decision
    if llm_output.get("tool"): # Use .get() for safer access
        tool_name = llm_output["tool"]
        tool_parameters = llm_output.get("parameters", {}) # Default to empty dict if no parameters

        print(f"Agent: Preparing to call tool '{tool_name}' with parameters: {tool_parameters}")

        tool_result = None
        try:
            # Map tool name to actual function
            if tool_name == "get_flight_status":
                tool_result = get_flight_status(**tool_parameters)
            elif tool_name == "get_weather_forecast":
                tool_result = get_weather_forecast(**tool_parameters)
            else:
                tool_result = {"error": f"Unknown tool: {tool_name}"}
        except TypeError as e:
            tool_result = {"error": f"Tool parameters mismatch for {tool_name}: {e}. Check LLM output."}
        except Exception as e:
            tool_result = {"error": f"Error executing tool {tool_name}: {e}"}


        print(f"Tool Result: {json.dumps(tool_result, indent=2)}")

        # 3. LLM can optionally process tool result for final response (refinement)
        # For simplicity, we'll just format the tool result here.
        if "error" in tool_result:
            final_response = f"I encountered an issue: {tool_result['error']}"
        elif tool_name == "get_flight_status":
            if tool_result["status"] == "Delayed":
                final_response = (f"Flight {tool_result['flight_number']} is {tool_result['status']} "
                                  f"from {tool_result['departure']} to {tool_result['arrival']}. "
                                  f"Scheduled: {tool_result['scheduled_departure_time']}, "
                                  f"Estimated: {tool_result['estimated_departure_time']} (Reason: {tool_result['delay_reason']}).")
            else:
                final_response = (f"Flight {tool_result['flight_number']} is {tool_result['status']} "
                                  f"from {tool_result['departure']} to {tool_result['arrival']}. "
                                  f"Scheduled departure: {tool_result['scheduled_departure_time']}.")
        elif tool_name == "get_weather_forecast":
            final_response = (f"The weather in {tool_result['location']} is {tool_result['conditions']} "
                              f"with a temperature of {tool_result['temperature']}. "
                              f"Humidity: {tool_result['humidity']}, Wind: {tool_result['wind']}. "
                              f"Forecast: {tool_result['forecast']}")
        else:
            final_response = f"Processed tool result: {json.dumps(tool_result, indent=2)}"

    else:
        # If LLM decided no tool was needed, or if an error occurred during Grok call
        final_response = llm_output.get("response", "I'm sorry, I couldn't process that request.")

    print(f"\nAgent Final Response: {final_response}")
    return final_response

# --- Demo Usage ---
if __name__ == "__main__":
    print("--- AI Agent with Grok 4 Tool Calling Demo ---")
    print("This demo simulates an AI agent using Grok 4 (if configured) to decide which tools to call.")
    print("All times are in EST, as per user preference.")

    # Example 1: Query requiring flight status tool
    print("\n\n--- Demo 1: Checking a flight status ---")
    ai_agent("What is the status of flight AC123?")

    # Example 2: Query requiring weather forecast tool (explicit location)
    print("\n\n--- Demo 2: Getting weather for a specific city ---")
    ai_agent("Tell me the weather forecast for Toronto.")

    # Example 3: Query requiring weather forecast tool (default location based on context)
    print("\n\n--- Demo 3: Getting weather for Montreal (default) ---")
    ai_agent("What's the weather like?") # Grok 4 should infer "Montreal" based on context/general knowledge if not explicitly provided

    # Example 4: Query for a delayed flight
    print("\n\n--- Demo 4: Checking a delayed flight ---")
    ai_agent("Is flight UA456 delayed?")

    print("\n\n--- Demo 5: Query for a flight not in mock data ---")
    ai_agent("What's the status of flight AA789?")

XAI SDK and Colab userdata imported successfully.
XAI Client and Grok 4 chat session initialized.
--- AI Agent with Grok 4 Tool Calling Demo ---
This demo simulates an AI agent using Grok 4 (if configured) to decide which tools to call.
All times are in EST, as per user preference.


--- Demo 1: Checking a flight status ---

Agent: Received Task: 'What is the status of flight AC123?'

LLM Reasoning: Sending prompt to Grok 4: 'What is the status of flight AC123?'
Grok 4 Raw Response: {"tool": "get_flight_status", "parameters": {"flight_number": "AC123"}}
Grok 4 parsed successfully. Tool: get_flight_status
Agent: Preparing to call tool 'get_flight_status' with parameters: {'flight_number': 'AC123'}

Tool Call: Calling 'get_flight_status' for flight: AC123
Tool Result: {
  "flight_number": "AC123",
  "status": "On Time",
  "departure": "Montreal (YUL)",
  "arrival": "Toronto (YYZ)",
  "scheduled_departure_time": "10:00 EST",
  "actual_departure_time": "10:00 EST"
}

Agent Final Response: 